# Test search utils


In [1]:
import yaml
import inspect
import importlib
import os,sys
import importlib
import arxiv

sys.path.append('..')

from model_discovery.evolution import BuildEvolution
import model_discovery.utils as U
import requests
import time
from tqdm import tqdm

headers={"X-API-KEY": os.environ['S2_API_KEY']}
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}'


strparams=[
    f"evoname=test_evo_000",
    "scales=14M,31M,70M",
    "selection_ratio=0.25",
    "select_method=random",
    "design_budget=0",
]
evo_system = BuildEvolution(
    strparams=';'.join(strparams),
    do_cache=False,
    # cache_type='diskcache',
)

ptree = evo_system.ptree


c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:1473: SyntaxWarning: invalid escape sequence '\('
  GUM_DESIGN_PROPOSER_SYSTEM_prompt = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:1698: SyntaxWarning: invalid escape sequence '\('
  GUM_PROPOSAL_REVIEWER_SYSTEM_prompt = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:2325: SyntaxWarning: invalid escape sequence '\('
  GUM_DESIGNER_SYSTEM_prompt_part4 = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:2631: SyntaxWarning: invalid escape sequence '\('
  GUM_IMPLEMENTATION_REVIEWER_SYSTEM_prompt = """
2024-09-13:14:26:36,627 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-13:14:26:37,678 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-13:14:26:37,936 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


{'evoname': 'test_evo_000', 'scales': '14M,31M,70M', 'selection_ratio': '0.25', 'select_method': 'random', 'design_budget': '0'}
Evolution system initialized with scales: ['14M', '31M', '70M']
Current scale: 0
Budgets remaining: {'70M': 1, '31M': 4, '14M': 16}
Checkpoint directory: C:\ChengJunyan1\Research\model_discovery\ckpt\test_evo_000
gpt2 tree loaded


2024-09-13:14:26:39,252 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-13:14:26:39,928 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-13:14:26:40,172 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


NameError: name 'SuperScholarSearcher' is not defined

## Get paper files

In [ ]:

refs_s2ids={}
REF_TYPES=['Reference','ReferenceCore','ReferenceWithCode','ReferenceCoreWithTree']

for i in ptree.G.nodes:
    node=ptree.G.nodes[i]['data']
    if node.type in REF_TYPES:
        s2id=node.s2id
        if s2id is not None:
            refs_s2ids[i]=s2id
        else:
            print(i)

refs=ptree.filter_by_type(REF_TYPES)

### Get rec and files

In [ ]:


def get_rec_papers(s2id):
    url=f'https://api.semanticscholar.org/recommendations/v1/papers/forpaper/{s2id}'
    rsp=requests.get(url,headers=headers)
    results=rsp.json()
    if 'error' in results:
        print(s2id)
        return None
    return results['recommendedPapers']

recs={}
for i in tqdm(refs_s2ids):
    s2id=refs_s2ids[i]
    rec=get_rec_papers(s2id)
    if rec is not None:
        recs[i]=rec
    time.sleep(0.1)

all_recs={}
for i in recs:
    for j in recs[i]:
        all_recs[j['paperId']]=j['title']

# all_recs_filtered={}
# for i in all_recs:
#     if i in refs_s2ids.values():
#         continue
#     if i in refs2_detail:
#         continue
#     all_recs_filtered[i]=all_recs[i]   


In [ ]:
 def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','plus')
refsp_detail={}
U.mkdir(save_dir)
for s2id in tqdm(all_recs_filtered):
    save_path=U.pjoin(save_dir,f'{s2id}.json')
    if U.pexists(save_path):
        refsp_detail[s2id]=U.load_json(save_path)
        continue
    ret=get_paper_detail(s2id)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refsp_detail[s2id]=ret
    time.sleep(0.1)


In [ ]:
len(refsp_detail)

In [ ]:
len(all_recs_filtered)

refp_arxiv={}
refp_pdfs={}
for i in refsp_detail:
    if 'openAccessPdf' in refsp_detail[i] and refsp_detail[i]['openAccessPdf'] is not None and 'arxiv' not in refsp_detail[i]['openAccessPdf']['url']:
        url=refsp_detail[i]['openAccessPdf']
        refp_pdfs[i]=url['url']
        continue
    if 'externalIds' in refsp_detail[i]:
        if 'ArXiv' in refsp_detail[i]['externalIds']:
            refp_arxiv[i]=refsp_detail[i]['externalIds']['ArXiv']
            continue
    # print(i)
print(len(refp_arxiv))
print(len(refp_pdfs))

In [ ]:
ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'
htmlsp_dir=U.pjoin(ptree.lib_dir,'..','files','htmlsp')
U.mkdir(htmlsp_dir)
pdfsp_dir=U.pjoin(ptree.lib_dir,'..','files','pdfsp')
U.mkdir(pdfsp_dir)

    
# for i in tqdm(refp_arxiv):
#     DIR=U.pjoin(htmlsp_dir,f'{i}.html')
#     if os.path.exists(DIR):
#         continue
#     url=ar5iv_url.format(arxiv_id=refp_arxiv[i])

#     # Send a GET request to the URL
#     response = requests.get(url)

#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Save the HTML content to a file
#         with open(DIR, "w", encoding="utf-8") as file:
#             file.write(response.text)
#     else:
#         print(f"Failed to retrieve {i}. Status code: {response.status_code}")



failed_pdfs={}
for i in tqdm(refp_pdfs):
    url=refp_pdfs[i]
    save_path=U.pjoin(pdfsp_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        try:
            pdf=urllib.request.urlretrieve(url, save_path)
        except:
            failed_pdfs[i]=url
            print(f'Failed to download {i}, {url}')



### Get references and files


In [ ]:
params={
    "fields": "openAccessPdf,externalIds",
}

pdf_urls={}
external_ids={}
for i in tqdm(refs_s2ids):
    s2id=refs_s2ids[i]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    if 'openAccessPdf' in results and results['openAccessPdf'] is not None:
        pdf_urls[i]=results['openAccessPdf']
    if 'externalIds' in results and results['externalIds'] is not None:
        external_ids[i]=results['externalIds']
    time.sleep(0.1)


arxiv_ids={}
for i,j in external_ids.items():
    if 'ArXiv' in j:
        arxiv_ids[i]=j['ArXiv']

arxiv_ids_add={
    'fire':'2310.04418',
    'mogrifier':'1909.01792',
    'hedgehog':'2402.04347',
    'synthesizer':'2005.00743',
    'settransformer':'1810.00825',
    'infiniteformer':'2109.00301',
    'etc':'2004.08483',
    'feedbackmem':'2002.09402',
    'kangpt':'2408.10205',
}

arxiv_ids.update(arxiv_ids_add)

ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'

In [ ]:
def get_node(id):
    return ptree.G.nodes[id]['data']


def get_references(id):
    params={
        "fields": "references",
    }
    s2id=refs_s2ids[id]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results['references']


all_refs={}
for i in tqdm(refs_s2ids):
    all_refs[i]=get_references(i)

refs_set={}
for i in all_refs:
    for ref in all_refs[i]:
        refs_set[ref['paperId']]=ref['title']


In [ ]:
def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,tldr,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

refs_detail={}
save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','refs2')
for i in tqdm(refs_set):
    save_path=U.pjoin(save_dir,f'{i}.json')
    if U.pexists(save_path):
        refs_detail[i]=U.load_json(save_path)
        continue
    ret=get_paper_detail(i)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refs_detail[i]=ret
    time.sleep(0.1)


In [ ]:
refs2_detail={}
ref2_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','secondary')
for i in os.listdir(ref2_dir):
    refs2_detail[i.split('.')[0]]=U.load_json(U.pjoin(ref2_dir,i))

ref2_arxiv={}
ref2_pdfs={}
for i in refs_detail:
    if 'externalIds' in refs2_detail[i]:
        if 'ArXiv' in refs2_detail[i]['externalIds']:
            ref2_arxiv[i]=refs2_detail[i]['externalIds']['ArXiv']
            continue
    if 'openAccessPdf' in refs2_detail[i] and refs2_detail[i]['openAccessPdf'] is not None and 'arxiv' not in refs2_detail[i]['openAccessPdf']['url']:
        url=refs_detail[i]['openAccessPdf']
        ref2_pdfs[i]=url['url']
        continue
    # print(i)

print(len(ref2_arxiv))
print(len(ref2_pdfs))

In [ ]:
import urllib, urllib.request

save_dir=U.pjoin(ptree.lib_dir,'..','pdfs2')
arxiv_pdf='https://arxiv.org/pdf/{arxiv_id}'
U.mkdir(save_dir)

# bar=tqdm(ref2_arxiv)
# for i in bar:
#     bar.set_description(f'Downloading {i}')
#     arxiv_id=ref2_arxiv[i]
#     download_arxiv_pdf(arxiv_id,save_dir,f'{i}.pdf')

failed_pdfs={}
for i in tqdm(ref2_pdfs):
    url=ref2_pdfs[i]
    save_path=U.pjoin(save_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        try:
            pdf=urllib.request.urlretrieve(url, save_path)
        except:
            failed_pdfs[i]=url
            print(f'Failed to download {i}, {url}')


In [ ]:
import requests

save_dir=U.pjoin(ptree.lib_dir,'..','htmls2')
U.mkdir(save_dir)
ar5iv_url='https://ar5iv.labs.arxiv.org/html/{arxiv_id}'

for i in tqdm(ref2_arxiv):
    DIR=U.pjoin(save_dir,f'{i}.html')
    if os.path.exists(DIR):
        continue
    url=ar5iv_url.format(arxiv_id=ref2_arxiv[i])
        
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the HTML content to a file
        with open(DIR, "w", encoding="utf-8") as file:
            file.write(response.text)
        # print(f"HTML content saved to {i}.html")
    else:
        print(f"Failed to retrieve {i}. Status code: {response.status_code}")


In [ ]:
htmls_dir=U.pjoin(ptree.lib_dir,'..','htmls')
pdfs_dir=U.pjoin(ptree.lib_dir,'..','pdfs')

htmls=[i.split('.')[0] for i in os.listdir(htmls_dir)]
pdfs=[i.split('.')[0] for i in os.listdir(pdfs_dir)]

print(len(htmls))
print(len(pdfs))

arxiv_url='https://arxiv.org/abs/{arxiv_id}'

for i in pdfs:
    if i not in htmls:
        if i in arxiv_ids:
            arxiv_id=arxiv_ids[i]
            url=ar5iv_url.format(arxiv_id=arxiv_id)
            arxiv=arxiv_url.format(arxiv_id=arxiv_id)
            print(url,arxiv,i)


In [ ]:
save_dir

In [ ]:
import urllib, urllib.request
url = 'http://export.arxiv.org/api/query?search_query="{query}"&start=0&max_results=1'
title = ptree.G.nodes[refs[0]]['data'].title
encoded_title = urllib.parse.quote(title)
url=url.format(query=encoded_title)
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))


In [ ]:
len(pdf_urls)

In [ ]:
import agents.search_utils 
importlib.reload(agents.search_utils)

from agents.search_utils import SuperScholarSearcher





# dev tools

In [ ]:
import importlib
import sys
# sys.path.append('..')

import model_discovery.agents.search_utils
importlib.reload(model_discovery.agents.search_utils)

from model_discovery.agents.search_utils import SuperScholarSearcher, pwc_search_patched

sss=SuperScholarSearcher(ptree,evo_system.stream)



2024-09-13:13:48:55,052 INFO     [discover_namespace_packages.py:12] Discovering subpackages in _NamespacePath(['c:\\Users\\ChengJunyan1\\anaconda3\\envs\\modis\\Lib\\site-packages\\pinecone_plugins'])
2024-09-13:13:48:55,055 INFO     [discover_plugins.py:9] Looking for plugins in pinecone_plugins.inference
2024-09-13:13:48:55,056 INFO     [installation.py:10] Installing plugin inference into PineconeGRPC


## Build Vector Store for internal Libs

In [ ]:
sss._build_vector_stores()

Loading splits Secondary:  64%|██████▎   | 1807/2836 [02:03<01:22, 12.54it/s]

Loading raw_texts in internal library...


Loading splits Secondary:  64%|██████▎   | 1807/2836 [02:15<01:22, 12.54it/s]2024-09-13:13:52:40,951 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-13:13:52:45,295 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Loading splits Secondary:  64%|██████▍   | 1808/2836 [03:16<3:25:09, 11.97s/it]2024-09-13:13:52:48,078 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-13:13:52:51,520 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Loading splits Secondary:  64%|██████▍   | 1809/2836 [03:22<3:04:59, 10.81s/it]2024-09-13:13:52:52,851 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-13:13:52:54,227 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-13:13:5

### Test Search 


In [20]:
grouped_docs,metainfo=sss.search_internal('hello')

2024-09-12:22:40:13,735 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-12:22:40:14,114 INFO     [_client.py:1038] HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


In [29]:
import numpy as np




In [30]:
print(ppr)

##### 1. LongT5: Efficient text-to-text transformer for long sequences (Avg. Score: 0.02)

*Mandy Guo, J. Ainslie, David C. Uthus, Santiago Ontañón, Jianmo Ni, Yun-Hsuan Sung, Yinfei Yang*

**TL;DR:** A new model, called LongT5, is presented, with which the effects of scaling both the input length and model size at the same time are explored, which mimics ETC's local/global attention mechanism, but without requiring additional side-inputs.

**Abstract:** Recent work has shown that either (1) increasing the input length or (2) increasing model size can improve the performance of Transformer-based neural models. In this paper, we present a new model, called LongT5, with which we explore the effects of scaling both the input length and model size at the same time. Specifically, we integrated attention ideas from long-input transformers (ETC), and adopted pre-training strategies from summarization pre-training (PEGASUS) into the scalable T5 architecture. The result is a new attention mecha